In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Задание 1
url = 'https://habr.com/ru/all/'

KEYWORDS = ['python', 'парсинг', 'frida', 'otus', 'angular', 'dart', 'линус']

df = pd.DataFrame()

def search_words(words: str) -> bool:
    words = words.lower()
    if any([desired in words for desired in KEYWORDS]):
        return True
    return False


def post_text(url_post: str) -> str:
    resp = requests.get(url_post)
    if resp.status_code == 200:
        soup = BeautifulSoup(resp.text, 'html.parser')
        return soup.find('div', class_='post__text').text, soup.find('span', class_='post__time').get('data-time_published')
    return


resp = requests.get(url)
if resp.status_code == 200:
    soup = BeautifulSoup(resp.text, 'html.parser')
    posts = soup.find_all('article', class_='post')

    for post in posts:
        search_id = False
        post_id = post.parent.attrs.get('id')
    # если идентификатор не найден, это что-то странное, пропускаем
        if not post_id:
            continue
        post_id = int(post_id.split('_')[-1])

        # ищем в заголовке
        titles = post.find_all('a', class_='post__title_link')
        for title in titles:
            if search_words(title.text):
                print(post_id, 'title')
                search_id = True
                break

        # ищем в хабах
        if not search_id:
            hubs = post.find_all('ul', class_='post__hubs')
            hub_lower = ''
            for hub in hubs:
                hub_lower += hub.text   
            search_id = search_words(hub_lower)

                
        # ищем в тексте
        if not search_id:
            texts = post.find_all('div', class_='post__text')
            for text in texts:
                if search_words(text.text):
                    search_id = True
                    break

        # ищем в статье
        if not search_id:
            title_element = post.find('a', class_='post__title_link')
            url_post = title_element.attrs.get('href')
            art_text, _ = post_text(url_post)    
            search_id = search_words(art_text)

        if search_id:
            title = post.find('a', class_='post__title_link')
            url_post = title.attrs.get('href')
            text, date = post_text(url_post)
            row = {'date': pd.to_datetime(date).date(), 'title': title.text, 'url': url_post, 'text': text}
            df = pd.concat([df, pd.DataFrame([row])])

df

,date,title,url,text
0,2020-12-09,Umka и трактор: первый опыт практического прим...,https://habr.com/ru/post/532018/,С весны нынешнего года я разрабатываю статичес...
0,2020-12-09,MikroTik Скрипт: Уведомление о успешном входе ...,https://habr.com/ru/post/532086/,Статья будет больше интересна специалистам исп...


In [7]:
# Задание 2
EMAIL = ['xxx@mail.ru', 'xxx@gmail.com', 'yyy@mail.ru']

df = pd.DataFrame(columns=['почта', 'дата утечки', 'описание утечки'])

url = "https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data"
payload = '{"emailAddresses":["' + '", "'.join(EMAIL) + '"]}'
headers = {
  'Content-Type': 'application/json;charset=UTF-8',
  'Vaar-Version': '0',
  'Vaar-Header-App-Product': 'hackcheck-web-avast',
  'Connection': 'keep-alive'
}

result = {}

response = requests.post(url, headers=headers, data=payload)
if response.status_code == 200:
  req = response.json()

  for br_val in req['breaches'].values():
    result[br_val['breachId']] = {}
    result[br_val['breachId']]['site'] = br_val['site']
    result[br_val['breachId']]['publishDate'] = br_val['publishDate']
    result[br_val['breachId']]['description'] = br_val['description']

  for site_val in req['data'].values():
    for mail, mail_vol in site_val.items():
      for br_id in mail_vol:
        result[br_id['breachId']]['mail'] = mail

  for br in result.values():
    row = {'почта': br['mail'], 'дата утечки': pd.to_datetime(br['publishDate']).date(), 'описание утечки': br['description']}
    df = pd.concat([df, pd.DataFrame([row])])


df.head(50)

,почта,дата утечки,описание утечки
0,yyy@mail.ru,2020-12-03,"In November 2020, a collection of over 23,000 ..."
0,xxx@gmail.com,2020-12-03,"In November 2020, a collection of over 23,000 ..."
0,xxx@gmail.com,2020-12-03,"In November 2020, a collection of over 23,000 ..."
0,xxx@gmail.com,2020-11-19,"At an unconfirmed date, the gym management sof..."
0,xxx@gmail.com,2020-12-03,"In November 2020, a collection of over 23,000 ..."
0,xxx@gmail.com,2020-07-30,"In March 2019, the Brazilian travel agency Her..."
0,xxx@gmail.com,2020-07-23,"In June 2020, the online writing community Wat..."
0,xxx@mail.ru,2020-07-02,"At an unconfirmed date, the French-language on..."
0,xxx@gmail.com,2020-08-20,"At an unconfirmed date, the Indian online tuto..."
0,xxx@gmail.com,2020-05-21,"At an unconfirmed date, the media search engin..."
